In [2]:
from aide_design.play import*
import math

# UASB Bottom and Effluent Geometry 
## Influent Distribution System


### Inlet Pipe Diameter

Two conflicting design requirements must be reconciled. 
1. The velocity at the top of the pipe should be low enough to allow any entrained air to escape back up the pipe
1. The velocity should be high enough at the outlet of the pipe to prevent heavy inert suspended solids from settling in or around the outlet.    

## Granular Sludge Characterization

Variability in granular sludge exists because of variations in wastewater composition and available substrate for biological activity, but granules generally comprise of 
1. inorganic material
1. extracellular polymer

Digested sludge was sourced from a methophilic anaerobic
digester (35 ± 1 C) of a municipal wastewater treatment plant
(WWTP) in Sendai, Japan. Anaerobic granular sludge was withdrawn
from sludge treatment plant effluent mixture of ethanol
and sugar factory waste water in Tokyo, Japan. The initial granular
sludge, consisting of well-settled black granules with about 80%
showing the size from 0.4 to 4.0 mm in diameter and 0.02 to
0.03 m/s of the sedimentation velocity, had a mixed liquor volatile
suspended solid (MLVSS) concentration of 28.5 g/L and mixed
liquor suspended solids (MLSS) concentration of 35.0 g/L. 

*(Operation performance and granule characterization of upflow
anaerobic sludge blanket (UASB) reactor treating wastewater with starch
as the sole carbon source
Xueqin Lu a
, Guangyin Zhen b
, Adriana Ledezma Estrada a
, Mo Chen c
, Jialing Ni a
, Toshimasa Hojo a
,
Kengo Kubota a
, Yu-You Li)*

In [5]:
diam_granule = 0.5 * u.mm               # Zoe Maisel
vel_settling_granule = 0.666 * u.mm/u.s # Zoe Maisel, experimental (Lu, 0.02 mm/s)
nu_water = 1 * (u.mm**2/u.s)            # viscosity
rho_water = 998 * (u.kg/u.m**3)         # density
phi_granule = 0.4                       # porosity

def rho_granule(vel_term, nu_water, rho_water, diam_min):
    rho_granule = (vel_term * 18 * nu_water * rho_water)/(pc.gravity * diam_min**2) + rho_water
    return rho_granule.to(u.kg/u.m**3)

rho_granule = rho_granule(vel_settling_granule, nu_water, rho_water, diam_granule)
print('The granule density is', rho_granule.to(u.kg/u.m**3))



The granule density is 1003 kilogram / meter ** 3


In [8]:
diam = 3 * u.ft
height = 7 * u.ft
    
WW_gen = 3 * u.mL/u.s        #Wastewater generated per person, rule of thumb from Monroe
WW_gen_bw = 0.6 * u.mL/u.s   #Assumes 20% of mixed wastewater
HRT = 4 * u.hr               #Hydraulic Residence Time, determined from lab scale tests
    
angle = 60 * u.deg 
height_cyl_hoof = diam/2 * np.tan(angle) 
vol_cyl_hoof = 2/3 * (diam/2)**2 * height_cyl_hoof
vol_reactor = (math.pi * (diam / 2)**2 * height / 2) - (2 * vol_cyl_hoof)
    
flow = vol_reactor / HRT
people_served = int(flow / WW_gen)       #People served per reactor
people_served_BW = int(flow / WW_gen_bw) #People served per reactor treating only blackwater
    
print(height_cyl_hoof)

2.598 foot


In [10]:
area_reactor = np.pi * diam**2 / 4   #May be erroneous because we do not have a circular active plan area

n_port = 4                           #Arbitrary
diam_port = 1 * u.inch               #Arbitrary
area_port = pc.area_circle(1 * u.inch)

vel_reactor = flow * area_reactor
vel_port = flow/n_port * area_port 

height_freefall = vel_port**2 / (2 * pc.gravity) # Does not account for minor losses from fittings, adapters, geometry 

